# Tests for quering local PBF osm files

guide for using the filter https://pypi.org/project/esy-osmfilter/


## imports

In [ ]:
import configparser, contextlib
import os, sys

from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation

import urllib.request


## Get file

In [ ]:
c = 'liechtenstein-latest'
city_name = 'berlin-latest'

PBF_inputfile = os.path.join(os.getcwd(), f'test-osmfilter/{city_name}.osm.pbf')
JSON_outputfile = os.path.join(os.getcwd(), f'test-osmfilter/{city_name}.json')

PBF_inputfile

In [ ]:
if not os.path.exists(f'test-osmfilter/{city_name}.osm.pbf'):
    filename, headers = urllib.request.urlretrieve(
        f'https://download.geofabrik.de/europe/{city_name}.osm.pbf',
        filename=f'test-osmfilter/{city_name}.osm.pbf')
# PBF_inputfile = filename

## Filter main data

In [ ]:
prefilter   = {Node: {}, Way: {"man_made":["pipeline",],}, Relation: {}}
blackfilter = [("pipeline","substation"),]
whitefilter = [(('man_made','pipeline'),('waterway','drain')),] #[(("waterway","drain"),("name","Wäschgräble")), ]

In [ ]:
>>> [Data,_]=run_filter('noname',
...                     PBF_inputfile, 
...                     JSON_outputfile, 
...                     prefilter,
...                     whitefilter, 
...                     blackfilter, 
...                     NewPreFilterData=True, 
...                     CreateElements=False, 
...                     LoadElements=False,
...                     verbose=True)

In [ ]:
>>> len(Data['Node'])

In [ ]:
>>> len(Data['Relation'])

In [ ]:
>>> len(Data['Way'])

In [ ]:
Data['Way']

## Filter out (blackfilter)

In [ ]:
>>> [_,Elements]=run_filter('funny-waterway-pipelines',
...                            PBF_inputfile, 
...                            JSON_outputfile, 
...                            prefilter,
...                            whitefilter, 
...                            blackfilter, 
...                            NewPreFilterData=False, 
...                            CreateElements=True, 
...                            LoadElements=False,
...                            verbose=True)
>>> len(Elements['funny-waterway-pipelines'])
0
>>> len(Elements['funny-waterway-pipelines']['Relation'])
0
>>> len(Elements['funny-waterway-pipelines']['Way'])

In [ ]:
Elements

## save geojson

In [ ]:
>>> from esy.osmfilter import export_geojson

# jsontype='Line' 'Point' 

>>> export_geojson(Elements['funny-waterway-pipelines']['Way'],Data,
... filename=f'test-osmfilter/{city_name}.geojson',jsontype='Line')

## open geosjon

In [ ]:
import geopandas as gpd

elements = gpd.read_file(f'test-osmfilter/{city_name}.geojson')
elements['geometry'][0]

In [ ]:
elements.describe()

# shapely, create geometry from geojson

In [ ]:
import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

# test

In [ ]:
green = {'landuse':['grass','forest','orchard','allotments','cementery','flowerbed', 'meadow','greenfield', 'recreation_ground','village_green'],
         'leisure':['park','playground','garden'],
         'natural':['heath','shrubbery','wood','grassland'],}
amenit = {'amenity':['bbq'],}

cycle = {'highway':['pedestrian','cycleway']}


prefilter   = {Node: {}, Way: cycle, Relation: {}}


[Data,_]=run_filter('noname',
                    PBF_inputfile, 
                    JSON_outputfile, 
                    prefilter,
                    whitefilter, 
                    blackfilter, 
                    NewPreFilterData=True, 
                    CreateElements=False, 
                    LoadElements=False,
                    verbose=True)

In [ ]:
ways = len(Data['Way'])
rels = len(Data['Relation'])
nodes = len(Data['Node'])
           
print(f"{ways} Ways")
print(f"{rels} Rels")
print(f"{nodes} Nodes")

In [ ]:
Data['Way']

In [ ]:
# jsontype='Line' 'Point' 

export_geojson(Data['Way'],Data,
filename=f'test-osmfilter/{city_name}.geojson',jsontype='Line')

## Create Shapely geomtry

In [ ]:
elements = gpd.read_file(f'test-osmfilter/{city_name}.geojson')
elements['geometry'][3]

In [ ]:
polygonize(elements['geometry'][1])

In [ ]:
lis = []
for l in elements['geometry']:
    lis.append(l)

In [ ]:
lis[2]

In [ ]:
type(lis[2])

In [ ]:
# converting to multi points

lis_p = geometry.MultiPoint(lis)


In [ ]:
# converting to multi lines

merged = linemerge([*lis])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
lines = geometry.MultiLineString(borders)

In [ ]:
lines

In [ ]:
# converting to mulpolygon

merged = linemerge([*lis])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
polygons = list(polygonize(borders))
parks = geometry.MultiPolygon(polygons)

In [ ]:
parks.geoms[3]

In [ ]:
type(parks)

## Create Geojson

In [ ]:
import folium
from shapely.geometry import mapping


In [ ]:
mapping(lines)

## example map in folium

In [ ]:
map_test = folium.Map(width=750,location=[52.5205,13.3988],height=500,tiles="openstreetmap",   zoom_start=11)
folium.GeoJson(mapping(lines)).add_to(map_test)

In [ ]:
map_test

# Final

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import configparser, contextlib
import os, sys

from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation

import urllib.request

import geopandas as gpd

import folium

import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

from shapely.geometry import mapping
from geojson import dump

In [2]:
from livablestreets.query_names_detailed import master_query

In [3]:
df = master_query()
df['geomtype'].iloc[0]

'Node'

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, public_transport_bus to water
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   query_string  22 non-null     object
 1   name          22 non-null     object
 2   distance      22 non-null     int64 
 3   geomtype      22 non-null     object
 4   jsontype      22 non-null     object
 5   shapelytype   22 non-null     object
 6   category      22 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.4+ KB


In [5]:
df['query_string'].iloc[0]

{'amenity': ['bus_station'], 'bus_bay': '', 'highway': ['bus_stop']}

## get file

https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf

In [6]:
def get_pbf_json(country,city):
    
    city_name = f'{city}-latest'
    
    if not os.path.exists(f'localfilter/{city_name}.osm.pbf'):
        filename, headers = urllib.request.urlretrieve(
            f'https://download.geofabrik.de/europe/{country}/{city_name}.osm.pbf',
            filename=f'localfilter/{city_name}.osm.pbf')
        PBF_inputfile = filename

    PBF_inputfile = os.path.join(os.getcwd(), f'localfilter/{city_name}.osm.pbf')
    JSON_outputfile = os.path.join(os.getcwd(), f'localfilter/{city_name}.json')

        
    return PBF_inputfile, JSON_outputfile

In [7]:
PBF_inputfile, JSON_outputfile = get_pbf_json('germany','berlin')

In [8]:
PBF_inputfile

'/home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.osm.pbf'

In [9]:
JSON_outputfile

'/home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.json'

## New heading

## execute filter

In [16]:
def init_filter(filter_name, prefilter):
    blackfilter = [("pipeline","substation"),]
    whitefilter = [(('man_made','pipeline'),('waterway','drain')),]
    Data, Elements=run_filter(filter_name,
                        PBF_inputfile, 
                        JSON_outputfile, 
                        prefilter,
                        whitefilter, 
                        blackfilter, 
                        NewPreFilterData=True, 
                        CreateElements=False, 
                        LoadElements=False,
                        verbose=True)

def shapely_format(lis):
    if shapelytype == 'Point':
        # converting to multi points
        shapely_geom = geometry.MultiPoint(lis)

    if shapelytype == 'lineString':
    # converting to multi lines
        merged = linemerge([*lis])  # merge LineStrings
        borders = unary_union(merged)  # linestrings to a MultiLineString
        shapely_geom = geometry.MultiLineString(lis)      

    if shapelytype == 'multiPolygon':
    #converting to multipolygon
        merged = linemerge([*lis])  # merge LineStrings
        borders = unary_union(merged)  # linestrings to a MultiLineString
        polygons = list(polygonize(borders))
        shapely_geom = geometry.MultiPolygon(lis)
        
    return shapely_geom


        
def run_filter(query_df):
    for index, row in query_df.iterrows():
        filter_name = index
        string = row['query_string']
        geometry = row['geomtype']
        
        if geometry == 'Node':
            query_string = {Node: string, Way: {}, Relation: {}}
        if geometry == 'Way':
            query_string = {Node: {}, Way: string, Relation: {}}

        jsontype = row['jsontype']
        category = row['category']
        shapelytype = row['shapelytype']
        
        #initilize filter
        data = init_filter(filter_name,query_string)
         
        export_geojson(Data[geometry],Data,filename=f'localfilter/{city_name}{filter_name}{category}.geojson',jsontype=jsontype)
        elements = gpd.read_file(f'test-osmfilter/{city_name}{filter_name}{category}.geojson')
            
        
        # create list of features
        lis = []
        for l in elements['geometry']:
            lis.append(l)
        
        shapely_geo = shapely_format(lis)

        
        gjson = mapping(shapely_geo)
        with open(f'{city_name}{filter_name}{category}.geojson', 'w') as f:
            dump(gjson, f)




In [17]:
run_filter(df)

TypeError: run_filter() got an unexpected keyword argument 'NewPreFilterData'

In [ ]:
df.index[0]

In [11]:
query_string = f'{{Node: {string}, Way: {{}}, Relation: {{}}}}'
prefilter = {Node: string, Way: {}, Relation: {}}

# init_filter(df.index[0],query_string)

In [15]:
string = df['query_string'].iloc[0]

prefilter = {Node: string, Way: {}, Relation: {}}
blackfilter = [("pipeline","substation"),]
whitefilter = [(('man_made','pipeline'),('waterway','drain')),]

Data, Elememt=run_filter(df.index[0],
                        PBF_inputfile, 
                        JSON_outputfile, 
                        prefilter,
                        whitefilter, 
                        blackfilter, 
                        NewPreFilterData=True, 
                        CreateElements=False, 
                        LoadElements=False,
                        verbose=True)

INFO:esy.osmfilter.pre_filter:PreFilter OSM GAS DATA
INFO:esy.osmfilter.pre_filter:InputFile     : /home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.osm.pbf
INFO:esy.osmfilter.pre_filter:Size          : 70008           kbyte
INFO:esy.osmfilter.pre_filter:Estimated Time: 10.00           s
INFO:esy.osmfilter.pre_filter:=============================
INFO:esy.osmfilter.pre_filter:0.5
INFO:esy.osmfilter.pre_filter:1
INFO:esy.osmfilter.pre_filter:1.3
INFO:esy.osmfilter.pre_filter:2
INFO:esy.osmfilter.pre_filter:3
INFO:esy.osmfilter.pre_filter:Outputfile    : /home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.json
Size          : 2778            kbyte 
Time Elapsed  : 19.92           s




In [18]:
Data

{'Node': {'27245242': {'id': 27245242,
   'tags': {'bench': 'no',
    'bin': 'no',
    'bus': 'yes',
    'highway': 'bus_stop',
    'lit': 'no',
    'name': 'Seeburger Chaussee',
    'public_transport': 'stop_position',
    'ref_name': 'Seeburg Seeburger Chaussee, Dallgow-Döberitz',
    'shelter': 'no',
    'tactile_paving': 'no'},
   'lonlat': [13.111422800000009, 52.51856560000007]},
  '29858077': {'id': 29858077,
   'tags': {'bench': 'yes',
    'bin': 'yes',
    'highway': 'bus_stop',
    'lit': 'yes',
    'name': 'Rebhuhnweg',
    'shelter': 'yes',
    'tactile_paving': 'no'},
   'lonlat': [13.558023599999986, 52.54483329999997]},
  '29858105': {'id': 29858105,
   'tags': {'bench': 'yes',
    'bin': 'yes',
    'highway': 'bus_stop',
    'lit': 'yes',
    'name': 'Rebhuhnweg',
    'shelter': 'yes',
    'tactile_paving': 'no'},
   'lonlat': [13.557721499999987, 52.545101799999976]},
  '30134871': {'id': 30134871,
   'tags': {'bench': 'yes',
    'bin': 'yes',
    'highway': 'bus_stop'